In [106]:
data = """FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
"""
data = open('puzzle.data').read()

def parse(data) -> tuple[ dict[complex, tuple[complex, complex]], complex ]:
    maze = {}
    for y, line in enumerate(data.splitlines()):
        for x, c in enumerate(line):
            pos = x + y * 1j
            connected_pos = {
                '|': (pos - 1j, pos + 1j),
                '-': (pos - 1, pos + 1),
                'L': (pos + 1, pos - 1j),
                'J': (pos - 1, pos - 1j),
                '7': (pos - 1, pos + 1j),
                'F': (pos + 1, pos + 1j)
            }
            if c in connected_pos:
                maze[pos] = connected_pos[c]
            elif c == 'S':
                maze[pos] = ('start',)
                start = pos
    return maze, start

def explore(maze: dict[complex, tuple[complex, complex]], path: list[complex]) -> list[complex]:
    open_list = [path]
    for path in open_list:
        for pos in maze[path[-1]]:
            if pos in maze and maze[pos] == ('start',) and len(path) > 2:
                return path + [pos]
            if pos in path or pos not in maze:
                continue
            open_list.append(path + [pos])
    return []

maze, start = parse(data)
for offset in [1, 1j, -1, -1j]:
    if start not in maze[start + offset]:
        continue
    path = explore(maze, [start, start + offset])
    if path:
        maze[start] = (path[1], path[-2])
        break

In [107]:
max_x = max(int(pos.real) for pos in maze)
max_y = max(int(pos.imag) for pos in maze)

inside_tile = 0
for y in range(max_y + 1):
    for x in range(max_x + 1):
        pos = x + y * 1j
        if pos in path:
            continue
        npos = x + y * 1j
        crossings = 0
        if npos == 1j:
            pass
        while npos.real <= max_x:
            if npos not in maze or npos not in path:
                npos += 1
                continue
            pipes = maze[npos]
            if npos - 1j in pipes and npos + 1j in pipes:
                crossings += 1
            elif (npos - 1j in pipes and npos + 1 in pipes) or (npos - 1 in pipes and npos + 1j in pipes):
                crossings += 0.5
            elif (npos + 1j in pipes and npos + 1 in pipes) or (npos - 1 in pipes and npos - 1j in pipes):
                crossings -= 0.5
            npos += 1
        inside_tile += crossings % 2
            
inside_tile


459.0